In [ ]:
# !pip install paramiko
# !pip install scp

In [ ]:
import numpy as np
import time
import datetime
import sys
from CloudComputing import CloudComputingApp
np.set_printoptions(threshold=sys.maxsize)

In [ ]:
cloudComputing = CloudComputingApp(instance_size=8)

In [ ]:
def task1(array_size, chunk_size, operation='addition'):
        print(f'Generating {array_size}x{array_size} matrix')
        arr = cloudComputing.generate_array(array_size, array_size)
        arr1 = cloudComputing.generate_array(array_size, array_size)
        print(f'Splitting {array_size}x{array_size} matrix into {chunk_size}x{chunk_size}')
        s_arr = cloudComputing.split_row(arr, chunk_size, chunk_size)
        s_arr1 = cloudComputing.split_row(arr1, chunk_size, chunk_size)
        print(f'Computing manual addition of {array_size}x{array_size} matrix')
        res1 = np.array(cloudComputing.matrix_add(arr, arr1))
        print(f'Manual computation done for {operation}')
        print(f"\nComputing distributed {operation} operation for {array_size}x{array_size}")
        # cloudComputing.get_queue(cloudComputing.sqs_client, )
        cloudComputing.compute_matrix_operation(operation, s_arr, s_arr1)
        # merge all the queue results
        print(f"\nMerging computation results")
        res2 = cloudComputing.merge_queue_result(s_arr, array_size, chunk_size)
        # cloudComputing.write_result_to_file(res2, f'{operation}_result.txt')
        #validating result
        print(f"\nVerifying computation results")
        if np.array_equal(res1, res2):
            print(f"Verification Successful!")
        else:
            print(f"Verification Failed!")

In [ ]:
def task2(array_size, chunk_size, operation='multiplication'):
        print(f'Generating {array_size}x{array_size} matrix')
        arr = cloudComputing.generate_array(array_size, array_size)
        arr1 = cloudComputing.generate_array(array_size, array_size)
        print(f'Splitting {array_size}x{array_size} matrix into {chunk_size}x{chunk_size}')
        s_arr = cloudComputing.split_row(arr, chunk_size, chunk_size)
        s_arr1 = cloudComputing.split_row(arr1, chunk_size, chunk_size)
        print(f'Computing manual {operation} of {array_size}x{array_size} matrix')
        res1 = np.array(cloudComputing.matrix_dot_product(arr, arr1))
        print(f"\nComputing distribution {operation} operation for {array_size}x{array_size}")
        cloudComputing.compute_matrix_operation(operation, s_arr, s_arr1)
        # wait for all the computation to finish
        time.sleep(cloudComputing.get_wait_time_based_on_matrix_size(array_size))
        # merge all the queue results
        print(f"\nMerging computation results")
        res2 = cloudComputing.merge_queue_result(s_arr, array_size, chunk_size)
        print(f"\nVerifying computation results")
        # cloudComputing.write_result_to_file(res2, f'{operation}_result.txt')
        #validating result
        print(f"\nVerifying computation results")
        if np.array_equal(res1, res2):
            print(f"Verification Successful!")
        else:
            print(f"Verification Failed!")

In [ ]:
cloudComputing.prepare_architecture()
# cloudComputing.purge_queue()

# TASK 1: Addition

In [ ]:
array_size = int(input("Please enter the array size: "))
chunk_size = int(input("Please enter the chunk size: "))

start_time = datetime.datetime.now()
task1(array_size=array_size, chunk_size=chunk_size, operation="addition")
print(f"Total computation time is {(datetime.datetime.now() - start_time)}")

# TASK 2: Multiplication

In [ ]:
array_size = input("Please enter the array size: ")
chunk_size = input("Please enter the chunk size: ")

start_time = datetime.datetime.now()
task2(array_size=array_size, chunk_size=chunk_size, operation="multiplication")
print(f"Total computation time is {(datetime.datetime.now() - start_time)}")

## Teardown Infrastructure

In [ ]:
cloudComputing.teardown_infrastructure()